Except where otherwise noted, this content is Copyright (c) 2020, [RTE](https://www.rte-france.com) and licensed under a [CC-BY-4.0 license](https://creativecommons.org/licenses/by/4.0/). <img src="https://mirrors.creativecommons.org/presskit/buttons/80x15/png/by.png" width="100px">

## What is a stochastic study ?
When you want to simulate a network adequacy, you can perform a deterministic computation. That means you believe you won’t have too much fluky behavior in the future. If you perform adequacy for the next hour or day, it’s a good hypothesis. But if you simulate network for the next week, month or year, it’s sound curious.

Are you sur wind will blow next week or sun will shines ? If not, you eolian or solar production could change. Can you warrant that no failure will occur on your network next month or next year ?

Of course, we can not predict future with such precision. It’s why we use stochastic computation. Stochastic means there are fluky behavior in the physics we want simulate. An single simulation is quiet useless, if result can change due to little variation.

The best solution could be to compute a *God function* which tell you for each input variation (solar production, line, consumptions) what is the adequacy result. Like that, Hadar has just to analyze function, its derivatives, min, max, etc to predict future. But this God function doesn’t exist, we just have an algorithm which tell us adequacy according to one fixed set of input data.

It’s why we use Monte Carlo algorithm. Monte Carlo run many scenarios to analyze many different behavior. Scenario with more consumption in cities, less solar production, less coal production or one line deleted due to crash. By this method we recreate God function by sampling it with the Monte-Carlo method.

<img src="monte-carlo.png">

## Describe example

We will reuse network seen in *Network Investment*. If you don't read this part, don't worry we just reuse network no more. It's look like

<img src="figure.png" width="70%">

We use data generated in the next topic [Workflow](https://www.hadar-simulator.org/tutorial?name=Workflow). Input data representes 10 scenarios with different load and eolien productions. There are also random faults for nuclear and gas. These 10 scenarios are unique. They are 10 random sampling on the *God function* to try to predict more widely network adequacy

In [1]:
import hadar as hd
import numpy as np

In [2]:
def read_csv(name):
    return np.genfromtxt('%s.csv' % name, delimiter=' ').T

In [3]:
line = 2000
study = hd.Study(['a', 'b', 'c', 'd'], horizon=168, nb_scn=10) \
    .add_on_node('a', data=hd.Consumption(name='load', cost=10**6, quantity=read_csv('load_A'))) \
    .add_on_node('a', data=hd.Production(name='gas', cost=80, quantity=read_csv('gas'))) \
    .add_link(src='a', dest='b', cost=5, quantity=line) \
\
    .add_on_node('b', data=hd.Consumption(name='load', cost=10**6, quantity=read_csv('load_B'))) \
    .add_link(src='b', dest='c', cost=5, quantity=line) \
\
    .add_on_node('c', data=hd.Production(name='nuclear', cost=50, quantity=read_csv('nuclear'))) \
    .add_link(src='c', dest='a', cost=5, quantity=line) \
    .add_link(src='c', dest='b', cost=10, quantity=line) \
    .add_link(src='c', dest='d', cost=10, quantity=line) \
\
    .add_on_node('d', data=hd.Consumption(name='load', cost=10**6, quantity=read_csv('load_D'))) \
    .add_on_node('d', data=hd.Production(name='eolien', cost=20, quantity=read_csv('eolien'))) \
    .add_link(src='d', dest='c', cost=10, quantity=line)

In [4]:
optimizer = hd.LPOptimizer()
res = optimizer.solve(study)

In [5]:
agg = hd.ResultAnalyzer(study, res)
plot = hd.HTMLPlotting(agg=agg, unit_symbol='MW', time_start='2020-06-19', time_end='2020-06-27',
                      node_coord={'a': [1.6264, 47.8842], 'b': [1.9061, 47.9118], 'c': [1.6175, 47.7097], 'd': [1.9314, 47.7090]})

Let's start by a quick overview of adequacy by plotting a remain available capacity. Blue squares mean network as enough energy to sustain consumption. Red square mean network has a lack of adequacy.

In [111]:
from ipywidgets import widgets, interactive_output
from IPython.display import display
import plotly.graph_objects as go

In [88]:
plot.network().map()

TypeError: map() missing 2 required positional arguments: 't' and 'zoom'

In [128]:
def dashboard(plotting):
    tabs = widgets.Tab()
    container = Container(tabs, plotting)
    nav = navbar(plotting.agg.study, container)
    return widgets.VBox([nav, tabs])

In [131]:
class Container:
    def __init__(self, tabs: widgets, plotting):
        self.tabs = tabs
        self.plotting = plotting
        self.update(nodes=None, types=None, names=None)
    
    def update(self, nodes, types, names):
        if nodes is None:
            self.network()
    
    def network(self):
        self.tabs.children = [self.rac(), self.exchanges()]
        self.tabs.set_title(0, 'RAC')
        self.tabs.set_title(1, 'Exchange Map')
        
    def rac(self):
        return go.FigureWidget(self.plotting.network().rac_matrix())
    
    def exchanges(self):
        def changes(time, scn, zoom):
            try:
                display(go.FigureWidget(self.plotting.network().map(t=time, scn=scn, zoom=zoom)))
            except ValueError:
                pass
        time = widgets.IntSlider(value=0, min=0, description='time', max=self.plotting.agg.horizon-1, continuous_update=False, disabled=False)
        scn = widgets.IntSlider(value=0, min=0, decription='scn', max=self.plotting.agg.nb_scn-1, continuous_update=False, disabled=False)
        zoom = widgets.FloatSlider(value=6, min=1, decription='zoom', max=10, disabled=False)
        hbox = widgets.HBox([time, scn, zoom])
        
        inter = interactive_output(changes, {'time': time, 'scn': scn, 'zoom': zoom})
        return widgets.VBox([hbox, inter])

In [132]:
dashboard(plot)

In [79]:
w = go.FigureWidget()

In [20]:
def navbar(study: hd.Study, tabs: Container):
    nodes = widgets.Dropdown(options=['All'] + list(study.nodes.keys()),
                             value='All', description='Nodes', disabled=False)
    types = widgets.Dropdown(options=['Node', 'Consumptions', 'Productions', 'Links'],
                             value='Node', description='elements', disabled=True)
    names = widgets.Dropdown(options=['None'], value='None', description='Names', disabled=True)
    
    def nodes_changes(state):
        if state['name'] == 'value' and state['type'] == 'change':
            if state['new'] == 'All':
                types.disabled = True
                names.disabled = True
                tabs.update(nodes=None, types=None, names=None)
            else:
                types.disabled = False
                types_changes(dict(name='value', type='change', new=types.value))    
    nodes.observe(nodes_changes)
    
    def types_changes(state):
        if state['name'] == 'value' and state['type'] == 'change':
            if state['new'] == 'Node':
                names.disabled = True
                tabs.update(nodes=nodes.value, types=None, names=None)
            else:
                if state['new'] == 'Consumptions':
                    el = [e.name for e in study.nodes[nodes.value].consumptions]
                elif state['new'] == 'Productions':
                    el = [e.name for e in study.nodes[nodes.value].productions]
                elif state['new'] == 'Links':
                    el = [e.name for e in study.nodes[nodes.value].links]
                names.options = el
                names.disabled = False
                names_changes(dict(name='value', type='change', new=names.value))
                
    types.observe(types_changes)
    
    def names_changes(state):
        if state['name'] == 'value' and state['type'] == 'change':
            tabs.update(nodes=nodes.value, types=types.value, names=names.value)
    names.observe(names_changes)
                
    return widgets.HBox([nodes, types, names])